# Introduction

**DALL-E** is a powerful AI model that can generate and edit images based on natural language prompts. **Image client** is a Python library that allows you to interact with DALL-E and other image models using a simple and intuitive interface. GPT-4 is a state-of-the-art language model that can understand and generate natural language or code. By combining these three components, you can create amazing applications that use natural language to generate and manipulate images. For example, you can use GPT-4 to generate a prompt for DALL-E, such as “a cat wearing a hat”, and then use image client to send the prompt to DALL-E and receive an image as a response. You can also use image client to edit the image, such as changing the color or shape of the hat, and then use GPT-4 to generate a description of the edited image. 

## Objectives:

1. Generate a prompt for DALL-E using GPT 3.5/4.0 model
2. Use image client to send the prompt to DALL-E and receive an image as a response.

## Import libraries

In [1]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import TransformChain, LLMChain, SequentialChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

## Get social media post from previous Lab

In [3]:
# Open the file
with open('social_media_post.txt', 'r') as f:
    social_media_post = f.read()

# Print the content
print(social_media_post)

Excited to share my latest episode of Behind the Tech podcast where I had an engaging conversation with none other than Neil deGrasse Tyson, one of America's most renowned astrophysicists and a gifted science communicator. We dove into the cosmos, discussing his passion for making complex scientific concepts clear and accessible. As the director of the Hayden Planetarium and the host of StarTalk Radio, Neil has a unique perspective on the universe and how we interact with it. Tune in for a fascinating discussion on astrophysics, science communication, and the role of technology in our world. #BehindTheTech #Podcast #Science #Technology


## Connect to Azure OpenAI service

In [13]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")

Found Azure OpenAI API Base Endpoint: https://swed-oai.openai.azure.com/


In [14]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

dalle_model = os.getenv("AZURE_OPENAI_DALLE_MODEL")
print(dalle_model)

gpt-4
None


## Use GPT model to generate prompt for DALL-E model

In [6]:
# Use GPT-4 to generate a DALL-E prompt
system_template="You are a helpful large language model that generates DALL-E prompts, that when given to the DALL-E model can generate beautiful high-quality images to use in social media posts about a podcast on technology.  Good DALL-E prompts will contain mention of related objects, and will not contain people or words.  Good DALL-E prompts should include a reference to podcasting along with items from the domain of the podcast guest.\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

user_prompt=PromptTemplate(
    template="Create a DALL-E prompt to create an image to post along with this social media text: {social_media_copy}",
    input_variables=["social_media_copy"],
)
human_message_prompt = HumanMessagePromptTemplate(prompt=user_prompt)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Get formatted messages for the chat completion
dalle_messages = chat_prompt.format_prompt(social_media_copy={social_media_post}).to_messages()

# Call Azure OpenAI Service to get a DALL-E prompt 
print("Calling GPT-4 model on Azure OpenAI Service to get a DALL-E prompt...\n")
gpt4 = AzureChatOpenAI(
    #openai_api_base=gpt4_endpoint,
    openai_api_version=api_version,
    deployment_name=model,
    #openai_api_key=gpt4_api_key,
    openai_api_type = "azure",
)
#print(gpt4)   #shows parameters

output = gpt4(dalle_messages)
dalle_prompt = output.content

dalle_prompt_chain = LLMChain(llm=gpt4, prompt=chat_prompt, output_key="dalle_prompt")

print("DALL-E Prompt:\n")
print(dalle_prompt)
print("\n")

Calling GPT-4 model on Azure OpenAI Service to get a DALL-E prompt...

DALL-E Prompt:

"A vintage microphone entwined with a spiral galaxy under a starlit sky, with a backdrop of the Hayden Planetarium and subtle elements of technology such as circuit boards and satellite dishes."




/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


## Save DALL-E prompt to file

In [7]:
# Specify the file path
file_path = "dalle_prompt.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(dalle_prompt)

In [8]:
dalle_endpoint = azure_endpoint
dalle_api_key = api_key

## Make a call to DALL-E model on the Azure OpenAI Service to generate an image 

In [8]:
print("Calling DALL-E model on Azure OpenAI Service to get an image for social media...\n")
# Append "high-quality digital art" to the generated DALL-E prompt
dalle_prompt = dalle_prompt + ", high-quality digital art"
print("dalleprompt: ",dalle_prompt)

Calling DALL-E model on Azure OpenAI Service to get an image for social media...

dalleprompt..................... "A vintage microphone entwined with a spiral galaxy under a starlit sky, with a backdrop of the Hayden Planetarium and subtle elements of technology such as circuit boards and satellite dishes.", high-quality digital art


In [9]:
from openai import AzureOpenAI
import os
import requests
from PIL import Image
import json

client = AzureOpenAI(
    api_version="2024-02-01",  
    api_key=api_key,  
    azure_endpoint=azure_endpoint
)

result = client.images.generate(
    model="dalle3", # the name of your DALL-E 3 deployment
    prompt=dalle_prompt,
    n=1
)

json_response = json.loads(result.model_dump_json())

# Initialize the image path (note the filetype should be png)
image_path = 'postImage.png'

# Retrieve the generated image
image_url = json_response["data"][0]["url"]  # extract image URL from response
generated_image = requests.get(image_url).content  # download the image
with open(image_path, "wb") as image_file:
    image_file.write(generated_image)

# Display the image in the default image viewer
image = Image.open(image_path)
image

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}